# Agent Based Model: Main Notebook

In [281]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [282]:
homogenous = pd.read_csv("Conditions/Homogenous.csv")
fiftyfifty = pd.read_csv("Conditions/FiftyFifty.csv")
heterogenous = pd.read_csv("Conditions/Heterogenous.csv")


for i in (range(len(homogenous.index))):
    print(list(homogenous.iloc[i][1:5]))


['ESFJ', 'ESFJ', 'ESFJ', 'ESFJ']
['ESTJ', 'ESTJ', 'ESTJ', 'ESTJ']
['ISFJ', 'ISFJ', 'ISFJ', 'ISFJ']
['ISTJ', 'ISTJ', 'ISTJ', 'ISTJ']
['ENFP', 'ENFP', 'ENFP', 'ENFP']
['ENFJ', 'ENFJ', 'ENFJ', 'ENFJ']
['INFP', 'INFP', 'INFP', 'INFP']
['INFJ', 'INFJ', 'INFJ', 'INFJ']
['ENTP', 'ENTP', 'ENTP', 'ENTP']
['ENTJ', 'ENTJ', 'ENTJ', 'ENTJ']
['INTP', 'INTP', 'INTP', 'INTP']
['INTJ', 'INTJ', 'INTJ', 'INTJ']
['ESFP', 'ESFP', 'ESFP', 'ESFP']
['ESTP', 'ESTP', 'ESTP', 'ESTP']
['ISFP', 'ISFP', 'ISFP', 'ISFP']
['ISTP', 'ISTP', 'ISTP', 'ISTP']


## Creating Study Groups

#### Practical Functions

In [283]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(1.5, 1.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [284]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [285]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ##
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension
        self.Type = Ex + Se + Th + Ju

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.5 + student.JuScore*0.5

In [286]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [287]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [288]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        Ind_Solution_attr_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.7) + 0.3, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
                Ind_Solution_attr_lst.append(true_solution[i])
            else:
                Ind_Solution_lst.append(random.randint(range_elements[0], range_elements[1]))
                Ind_Solution_attr_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_attr_lst
    
    return all_solutions

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [289]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                continue
            if student.Type != eval(presenter_name).Type: # evaluate whether presenter is same type as the student. If they are not, we do a coinflip determining whether or not they will be presented to
                coin_toss = np.random.binomial(1, 0.5, 1)[0]
                if coin_toss == 1:
                    pass
                else:
                    continue
                
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, ((1-student.ThScore)*0.2 + student.ExScore*0.1 + eval(presenter_name).Academic_Skill*0.1), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            break



    #print("Number of iterations: " + str(n_ticks))
    return individual_solutions[0], n_ticks # returns their collective solution


In [290]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


In [291]:
def adaptation_degree(individual_solution, group_solution):
    '''
    #function to calculate the adaptation degree of the agents
    '''
    adapted = 0
    not_adapted = 0

    for i in range(len(individual_solution)):
        if individual_solution[i] == group_solution[i]:
            not_adapted += 1
        else:
            adapted += 1
    
    adaptation = adapted / (adapted + not_adapted)

    return adaptation

In [292]:
def synergy_calculator(individual_accuracies, group_accuracy):
    '''
    #function to calculate the gain of the agents
    '''
    avg_of_indi_accuracy = np.mean(individual_accuracies)
    max_of_indi_accuracy = np.max(individual_accuracies)
    
    weak_synergy = group_accuracy - avg_of_indi_accuracy
    strong_synergy = group_accuracy - max_of_indi_accuracy

    return weak_synergy, strong_synergy

# TEST RUN

In [293]:
# Selecting Personality Types
student_list_test = ["INTP", "INTP", "INTP", "INTP"]

In [294]:
# Creating the Study Group
studygroup1 = StudyGroup(student_list_test)

In [295]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  INTP  0.374325  0.192729  0.520660  0.273174  0.540222
1    Bravo  INTP  0.473344  0.402229  0.627895  0.483096  0.540433
2  Charlie  INTP  0.361329  0.022791  0.756171  0.131500  0.554354
3    Delta  INTP  0.449313  0.365831  0.784716  0.178862  0.406516


In [296]:
# Generating a True Solution
true_solution_test = true_solution_generator(10, [1, 3])

In [297]:
# Displaying the True Solution
true_solution_test[0]

[2, 2, 1, 2, 2, 2, 1, 3, 1, 1]

In [298]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[2, 1, 1, 1, 2, 2, 1, 3, 1, 2],
 [2, 1, 1, 2, 2, 3, 1, 3, 1, 2],
 [2, 1, 1, 2, 3, 2, 1, 3, 1, 1],
 [2, 3, 1, 2, 1, 2, 3, 1, 1, 1]]

In [299]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [2, 2, 1, 1, 2, 2, 1, 3, 1, 3]
Bravo [2, 3, 1, 2, 2, 2, 1, 2, 1, 3]
Charlie [2, 2, 1, 2, 1, 1, 1, 3, 1, 1]
Delta [2, 3, 1, 1, 2, 3, 1, 2, 1, 1]


In [300]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 0.8
Bravo 0.7
Charlie 0.8
Delta 0.6


In [301]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [302]:
print(group_solution)
print(all_solutions)

([2, 2, 1, 2, 2, 2, 1, 2, 1, 1], 24)
[[2, 1, 1, 1, 2, 2, 1, 3, 1, 2], [2, 1, 1, 2, 2, 3, 1, 3, 1, 2], [2, 1, 1, 2, 3, 2, 1, 3, 1, 1], [2, 3, 1, 2, 1, 2, 3, 1, 1, 1]]


In [303]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

0.9


In [304]:
adaptation_degree(all_solutions[0], group_solution[0])

0.4

## ABM Model

In [305]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    #### ---- LISTS ---- #### 
    ## ---- group level ---- ###
    group_accuracy_list = []
    n_tick_list = []
    weak_synergy_list = []
    strong_synergy_list = []

    ## ---- individual level ---- ###
    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []

    alfa_adaptation_list = []
    beta_adaptation_list = []
    charlie_adaptation_list = []
    delta_adaptation_list = []

    # ---- starting simulation ---- #
    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)
        
        ## ---- SOLUTIONS ---- ##
        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])

        all_accuracies = [alfa_accuracy, beta_accuracy, charlie_accuracy, delta_accuracy]
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        
        ## ---- MEASURES ---- ##
        ## calculating degree of adaptation ## 
        alfa_adaption = adaptation_degree(all_solutions[0], group_solution[0])
        beta_adaption = adaptation_degree(all_solutions[1], group_solution[0])
        charlie_adaption = adaptation_degree(all_solutions[2], group_solution[0])
        delta_adaption = adaptation_degree(all_solutions[3], group_solution[0])

        alfa_adaptation_list.append(alfa_adaption)
        beta_adaptation_list.append(beta_adaption)
        charlie_adaptation_list.append(charlie_adaption)
        delta_adaptation_list.append(delta_adaption)

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

        # calculating the gain from the individual solutions to the group solution
        weak_synergy, strong_synergy = synergy_calculator(all_accuracies, group_accuracy)
        weak_synergy_list.append(weak_synergy)
        strong_synergy_list.append(strong_synergy)
        

    

    ## calculating the mean and standard deivations ##
    # ---- group level ---- #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)

    avg_weak_synergy  = np.mean(weak_synergy_list)
    std_weak_synergy = np.std(weak_synergy_list)

    avg_strong_synergy  = np.mean(strong_synergy_list)
    std_strong_synergy = np.std(strong_synergy_list)
    
    # ---- individual level ---- #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)

    avg_alfa_adaptation = np.mean(alfa_adaptation_list)
    avg_beta_adaptation = np.mean(beta_adaptation_list)
    avg_charlie_adaptation = np.mean(charlie_adaptation_list)
    avg_delta_adaptation = np.mean(delta_adaptation_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_weak_synergy': avg_weak_synergy,
    'std_weak_synergy': std_weak_synergy, 
    'avg_strong_synergy': avg_strong_synergy,
    'std_strong_synergy': std_strong_synergy}

    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy], 
                                    [avg_alfa_adaptation, avg_beta_adaptation, avg_charlie_adaptation, avg_delta_adaptation]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [306]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_weak_synergy', 'std_weak_synergy', 'avg_strong_synergy', 'std_strong_synergy'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df, condition_group_df


In [307]:
homogenous_ind_df, homogeous_group_df = abm_model(condition = homogenous, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [308]:
sorted_wsynergy_homogenous = homogeous_group_df.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_homogenous

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
3,ISTJ,ISTJ,ISTJ,ISTJ,0.656,0.147865,35.08,15.540708,-0.03050,0.128335,-0.193,0.151165
14,ISFP,ISFP,ISFP,ISFP,0.552,0.153935,24.70,9.196195,-0.00750,0.137545,-0.175,0.158981
12,ESFP,ESFP,ESFP,ESFP,0.552,0.143164,19.13,7.876109,-0.00550,0.125727,-0.166,0.138000
5,ENFJ,ENFJ,ENFJ,ENFJ,0.815,0.128355,13.02,6.514568,-0.00450,0.112104,-0.130,0.119583
9,ENTJ,ENTJ,ENTJ,ENTJ,0.818,0.125204,18.50,8.522324,-0.00275,0.113737,-0.122,0.124563
13,ESTP,ESTP,ESTP,ESTP,0.541,0.146352,36.10,15.997812,-0.00225,0.137254,-0.167,0.172659
8,ENTP,ENTP,ENTP,ENTP,0.686,0.138578,28.03,14.972278,-0.00025,0.129735,-0.164,0.157175
11,INTJ,INTJ,INTJ,INTJ,0.827,0.134800,26.17,12.983108,0.00175,0.114687,-0.119,0.133937
2,ISFJ,ISFJ,ISFJ,ISFJ,0.682,0.149920,19.78,7.542652,0.00500,0.129180,-0.169,0.148118
7,INFJ,INFJ,INFJ,INFJ,0.820,0.131909,15.05,7.933946,0.00550,0.115790,-0.123,0.127165


In [309]:
fifty_ind, fifty_group = abm_model(condition = fiftyfifty, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [310]:
sorted_wsynergy_fifty = fifty_group.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_fifty

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
6,ESTP,ESTP,INFJ,INFJ,0.625,0.169926,40.87,16.668926,-0.06350,0.154006,-0.269,0.187454
9,ENTP,ENTP,INFJ,INFJ,0.724,0.138651,30.57,14.640529,-0.03825,0.136769,-0.200,0.159374
14,ESTP,ESTP,ISFJ,ISFJ,0.611,0.153555,40.42,18.329855,-0.01450,0.144490,-0.196,0.176590
10,ESFP,ESFP,INTP,INTP,0.597,0.162145,59.02,31.635733,-0.01325,0.141795,-0.205,0.174571
13,ENFP,ENFP,INTJ,INTJ,0.750,0.142478,41.03,20.985450,-0.00675,0.128834,-0.171,0.149863
5,ENFP,ENFP,ISTJ,ISTJ,0.683,0.140752,48.38,22.111888,-0.00575,0.130808,-0.164,0.148000
0,ESFJ,ESFJ,INTP,INTP,0.691,0.149060,49.87,22.511177,0.00425,0.119271,-0.155,0.156445
1,ENTP,ENTP,ISFJ,ISFJ,0.697,0.150635,35.73,16.600515,0.00475,0.140566,-0.147,0.145914
2,ESFP,ESFP,INTJ,INTJ,0.685,0.117792,54.29,29.118824,0.00575,0.124542,-0.203,0.146598
4,ESTJ,ESTJ,INFP,INFP,0.698,0.147635,36.40,17.073371,0.01075,0.127586,-0.140,0.143527


In [311]:
hetero_ind, hetero_group = abm_model(condition = heterogenous, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [312]:
sorted_wsynergy_hetero = hetero_group.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_hetero

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
11,ENFJ,INTP,INFJ,ENTP,0.744,0.144444,45.88,21.370671,-0.01475,0.131581,-0.173,0.144121
5,ESTJ,ESTP,ISTP,INTJ,0.645,0.151905,76.93,36.613455,-0.01450,0.139471,-0.221,0.167508
0,ESFJ,ESTJ,ENFP,INFJ,0.715,0.148577,40.46,18.733083,-0.01000,0.131339,-0.166,0.138723
14,ESTJ,ISTJ,ENFJ,INFJ,0.745,0.151905,42.99,17.282647,-0.00950,0.140969,-0.172,0.157531
1,ISFJ,ESTP,ISFP,ENFJ,0.655,0.168152,44.65,18.310311,-0.00725,0.147788,-0.208,0.169517
4,ESFJ,ISFJ,ISTJ,INTP,0.688,0.155100,56.21,28.146152,-0.00225,0.140494,-0.145,0.151245
8,ESFJ,ISTP,ESTP,ENTP,0.627,0.139180,64.79,30.681687,0.00125,0.121417,-0.160,0.132665
9,ESTJ,ISFP,ISTJ,ESFP,0.625,0.153216,60.36,27.864142,0.00450,0.149390,-0.174,0.165300
3,INTP,INTJ,INFP,ENTP,0.734,0.160137,58.70,27.527986,0.00875,0.133387,-0.153,0.145914
13,ENTP,INTP,ESFP,ISFP,0.634,0.144375,59.41,24.231011,0.01325,0.130310,-0.165,0.129904


In [313]:
homogenous_mean_acc = np.mean(homogeous_group_df['avg_accuracy'])
fifty_mean_acc = np.mean(fifty_group['avg_accuracy'])
hetero_mean_acc = np.mean(hetero_group['avg_accuracy'])

homogenous_std_acc = np.std(homogeous_group_df['avg_accuracy'])
fifty_std_acc = np.std(fifty_group['avg_accuracy'])
hetero_std_acc = np.std(hetero_group['avg_accuracy'])

homogenous_mean_tick = np.mean(homogeous_group_df['avg_n_tick'])
fifty_mean_tick = np.mean(fifty_group['avg_n_tick'])
hetero_mean_tick = np.mean(hetero_group['avg_n_tick'])

homogenous_std_tick = np.std(homogeous_group_df['avg_n_tick'])
fifty_std_tick = np.std(fifty_group['avg_n_tick'])
hetero_std_tick = np.std(hetero_group['avg_n_tick'])

print(tuple([homogenous_mean_acc, fifty_mean_acc, hetero_mean_acc]))
print(tuple([homogenous_std_acc, fifty_std_acc, hetero_std_acc]))

print(tuple([homogenous_mean_tick, fifty_mean_tick, hetero_mean_tick]))
print(tuple([homogenous_std_tick, fifty_std_tick, hetero_std_tick]))

(0.6896875, 0.6945000000000001, 0.69925)
(0.09469801921766899, 0.055199637679970395, 0.05442253669207272)
(24.899375000000003, 42.1675, 52.810625)
(9.32718638493812, 8.859995414784365, 12.557460665252949)
